## Supervised Learning - Classification


### Binary classification of protein sequences

Proteins are polymers made of 20 different amino acids. Proteins have been classified into different families based on their sequence similarity. 
Positive and negative datasets corresponding to one of the protein family are available at http://www.imtech.res.in/raghava/icaars/supplementary.html

### Extract feature from the protein sequences 
Amino Acid Composition refers to frequency of each amino acid within a protein sequence. E.g. if a protein has a sequence 'MSAARQTTRKAE' it's amino acid composition can be represented as a vector of length 20:

'A':3,'C':0,'D':0,'E':1,'F':0,'G':0,'H':0,'I':0,'K':1,'L':0,'M':1,'N':0,'P':0,'Q':1,'R':1,'S':1,'T':2,'V':0,'W':0,'Y':0

In this way all the protein sequences can be represented as feature vector of contant length. Note that protein sequences within the same class can have different number of amino acids.

In [ ]:
import scipy
import numpy as np
import pandas as pd
from Bio import SeqIO
from Bio.SeqUtils.ProtParam import ProteinAnalysis
positive_dict = SeqIO.to_dict(SeqIO.parse("positive-aars.txt", "fasta")) ##fasta is a type of sequence format
negative_dict = SeqIO.to_dict(SeqIO.parse("negative-aars.txt", "fasta"))

## Amino acid composition calculation##
#c1 = ProteinAnalysis("AAAASTRRRTRRAWEQWERQW").count_amino_acids()
df1 = pd.DataFrame()
for keys,values in positive_dict.iteritems():
    df1 = df1.append(pd.Series(ProteinAnalysis(str(values.seq)).get_amino_acids_percent(),name='1'))
for keys,values in negative_dict.iteritems():
    df1 = df1.append(pd.Series(ProteinAnalysis(str(values.seq)).get_amino_acids_percent(),name='-1'))

df1

### Plot features

In [ ]:
%matplotlib inline
import seaborn as sns; sns.set()
df1['index1'] = [int(x) for x in df1.index.values]
#sns.pairplot(df1, hue='index1', palette="husl", size=1.5); ##Slow
sns.heatmap(df1.corr())

### Model Selection

In [ ]:
from sklearn import model_selection
# Split-out validation dataset\n",
labels = df1.index.values
df1_matrix = df1.iloc[:,range(20)].as_matrix().astype(np.float)

validation_size = 0.20
seed = 7
X_train, X_validation, Y_train, Y_validation = model_selection.train_test_split(df1_matrix, labels, test_size=validation_size, random_state=seed)


In [ ]:
# from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

# Test options and evaluation metric
seed = 7
scoring = 'accuracy'

# Spot Check Algorithms
models = []
models.append(('LR', LogisticRegression()))
models.append(('LDA', LinearDiscriminantAnalysis()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('CART', DecisionTreeClassifier()))
models.append(('NB', GaussianNB()))
models.append(('SVM', SVC()))
# evaluate each model in turn
results = []
names = []
for name, model in models:
    kfold = model_selection.KFold(n_splits=5, random_state=seed)
    cv_results = model_selection.cross_val_score(model, X_train, Y_train, cv=kfold, scoring=scoring)
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)

#### Check the hyperparameters for the SVM model

In [ ]:
SVC().get_params()

#### Build SVM model with different set of hyperparameters 

In [ ]:
from sklearn.svm import SVC
import sklearn
clf_poly = SVC(kernel='poly', degree=3)
clf_rbf = SVC(kernel='rbf', C=10)

kfold = model_selection.KFold(n_splits=5, random_state=seed)
cv_SVM_poly_results = model_selection.cross_val_score(clf_poly, X_train, Y_train, cv=kfold, scoring=scoring)
cv_SVM_rbf_results = model_selection.cross_val_score(clf_rbf, X_train, Y_train, cv=kfold, scoring=scoring)
print (cv_SVM_poly_results.mean(), cv_SVM_rbf_results.mean())

### Exercise: Find a set of hyperparameters for SVM model that can give accuracy of greater than fifty percent

In [ ]:
## Solution
model1 = SVC() #add hyperparameters

kfold = model_selection.KFold(n_splits=5, random_state=seed)
cv_model1 = model_selection.cross_val_score(model1, X_train, Y_train, cv=kfold, scoring=scoring)
print (cv_model1.mean())

#### Read documentaion to select the right combination of hyperparameters associated with a particular model. 
Choice of some hyperparameters could affect the interpretation of other hyperparameters for a given model. E.g. in case of SVM model the `degree` parameter is applicable only if the kernel is `poly`. The value for `degree` is ignored for all other kernels. Similarly `gamma` is not applicable is case of `linear` kernel.

In [ ]:
clf_linear?

### Grid Search

In [ ]:
from sklearn.model_selection import GridSearchCV
tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],
                     'C': [1, 10, 100, 1000, 10000]},
                    {'kernel': ['linear'], 'C': [1, 10, 100, 1000, 10000]},
                    {'kernel':['poly'], 'C': [1, 10, 100, 1000, 10000],
                     'degree': range(10)}]
clf_grid = GridSearchCV(SVC(), tuned_parameters, cv=5, scoring='accuracy')
clf_grid.fit(df1_matrix,labels)


In [ ]:
print(clf_grid.best_params_)
clf_grid.cv_results_;

#### Build a SVM model using above hyperparameters and check the prediction accuracy

In [ ]:
clf_linear = SVC(kernel='linear',C=1000)

kfold = model_selection.KFold(n_splits=5, random_state=seed)
cv_SVM_results = model_selection.cross_val_score(clf_linear, X_train, Y_train, cv=kfold, scoring=scoring)
print (cv_SVM_results.mean(),cv_results.std())

### Confusion Matrix

In [ ]:
from sklearn.svm import SVC
from collections import Counter
import sklearn
clf_linear = SVC(kernel='linear',C=1000)

labels = df1.index.values
df1_matrix = df1.iloc[:,range(20)].as_matrix().astype(np.float)
print(df1_matrix.shape)
## Fit model to the data ##
clf_linear.fit(df1_matrix,labels)
print(Counter(labels))

## Predict labels for the original data ##
clf_linear_predict = clf_linear.predict(df1_matrix)
Counter(clf_linear_predict)
#print(clf_linear_predict)


In [ ]:
from sklearn.metrics import confusion_matrix
import matplotlib as plt
import seaborn as sn
cm = confusion_matrix(labels,clf_linear_predict)
print(cm)

## Plot ##
sns.heatmap(cm, square=True, annot=True, cbar=False)


## Exercise

#### Extract another feature - Di-Peptide Composition (DPC)

For each sequence calculate the frequency of pairwaise occurrence of amino acids. The length of the feature vector for each sequence would be 400 (20 x 20). Construct a classification model using DPC as feature.

In [ ]:
import itertools
aa_list = ['A','C','D','E','F','G','H','I','K','L','M','N','P','Q','R','S','T','V','W','Y']


In [ ]:
## Create a series of dipeptides
dpc_series = pd.Series(name='1')
for x in itertools.product(aa_list,aa_list):
    dpc_series[''.join([''.join(x)])] = 0
    

Calculate dipeptide composition for the positive dataset.

In [ ]:
df_dpc = pd.DataFrame([])
for keys,values in positive_dict.iteritems():
    dpc_series_copy = dpc_series.copy()
#    print (values.seq)
    dpc_seq = [str(values.seq[i:i+2]) for i in range(len(values.seq))]
    del dpc_seq[-1]
    for x in dpc_seq:
        dpc_series_copy[x] += 1
    dpc_series_copy /= len(values.seq)
    dpc_series_copy *= 100
    df_dpc = df_dpc.append(dpc_series_copy)
#dpc_series_copy

In [ ]:
df_dpc

#### Similarly, calculate the dipeptide composition for the negative dataset and append to the dataframe

In [ ]:
for keys,values in negative_dict.iteritems():
    dpc_series_copy = dpc_series.copy()
    dpc_series_copy.name = '-1'
    dpc_seq = [str(values.seq[i:i+2]) for i in range(len(values.seq))]
    del dpc_seq[-1]
    for x in dpc_seq:
        dpc_series_copy[x] += 1
    dpc_series_copy /= len(values.seq)
    dpc_series_copy *= 100
    df_dpc = df_dpc.append(dpc_series_copy)
df_dpc

In [ ]:
from sklearn import model_selection
labels = df_dpc.index.values
df_dpc_matrix = df_dpc.iloc[:,range(20)].as_matrix().astype(np.float)

validation_size = 0.20
seed = 7
X_train, X_validation, Y_train, Y_validation = model_selection.train_test_split(df_dpc_matrix, labels, test_size=validation_size, random_state=seed)


In [ ]:
from sklearn.svm import SVC
clf_linear = SVC(kernel='linear',C=1000)

kfold = model_selection.KFold(n_splits=5, random_state=seed)
cv_SVM_results = model_selection.cross_val_score(clf_linear, X_train, Y_train, cv=kfold, scoring='accuracy')
print (cv_SVM_results.mean(),cv_SVM_results.std())

### Exercise: Find the best set of hyperparameters for SVM model that can be used for classification of proteins using dipeptide composition feature

In [1]:
# Solution [Hint: Use grid search]
